# API de Gerenciamento de Imagens - Teste com Usuário Mock Personalizado

Este notebook permite testar a API de imagens com um usuário mock personalizado, resolvendo problemas de integridade referencial relacionados ao team_id.

In [12]:




# Importação das bibliotecas necessárias
import requests
import json
import uuid
import io
import os
from IPython.display import display, HTML
from PIL import Image as PILImage
import base64
import re

In [13]:
# Configuração base da API
BASE_URL = "http://localhost:8080"  # Ajuste conforme necessário
API_KEY = "imapi_demo_key_for_testing_purposes_only"  # Ajuste conforme necessário

# Configuração para substituir o mock user
# IMPORTANTE: Substitua este team_id por um que existe no banco de dados!
MOCK_CONFIG = {
    "username": "daniel.schulz_computer_vision_team",
    "email": "daniel.schulz_computer_vision_team@sereact-ai.com",
    "team_id": "2e07cc40-68e2-40cc-b686-cb05dce6c408",
    "id": "37c9e956-3105-4846-bc81-261eb6094d2c",
    "created_at": "2025-05-30T20:27:00.608435+00:00",
}

## 3. Funções de Suporte para Chamadas à API

In [14]:
def make_api_request(method, endpoint, headers=None, params=None, json_data=None, files=None, data=None):
    """Função base para fazer requisições à API, aplicando o mock user"""
    url = f"{BASE_URL}{endpoint}"
    
    # Preparar cabeçalhos com autenticação e x-mock-user
    if headers is None:
        headers = {}
    
    # Adicionar chave de API se não estiver presente
    if "Authorization" not in headers:
        headers["Authorization"] = API_KEY
    
    # Adicionar mock user configuration
    mock_header = base64.b64encode(json.dumps(MOCK_CONFIG).encode()).decode()
    headers["X-Mock-User"] = mock_header
    
    # Se estamos enviando dados de formulário, adicionar team_id explicitamente
    if data is None:
        data = {}
    
    # Adicionar o team_id explicitamente nos dados
    data["team_id"] = MOCK_CONFIG["team_id"]
    
    print(f"Requisição: {method} {url}")
    print(f"Usando mock user: {MOCK_CONFIG['username']} (Team ID: {MOCK_CONFIG['team_id']})")
    
    # Fazer a requisição
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        params=params,
        json=json_data,
        files=files,
        data=data
    )
    
    print(f"Status: {response.status_code}")
    
    # Tentar analisar a resposta como JSON
    try:
        result = response.json()
        print("Resposta:")
        print(json.dumps(result, indent=2)[:500])
        if len(json.dumps(result, indent=2)) > 500:
            print("... (truncado)")
        return result
    except:
        # Não é JSON, retorna o texto
        print(f"Resposta (não JSON):\n{response.text[:200]}")
        if len(response.text) > 200:
            print("... (truncado)")
        return response.text

## 5. Criar Função de Upload de Imagem com Mock User

In [17]:
def create_test_image(color, width=300, height=200):
    """Cria uma imagem de teste com cor sólida"""
    if color == "red":
        rgb = (255, 0, 0)
    elif color == "green":
        rgb = (0, 255, 0)
    elif color == "blue":
        rgb = (0, 0, 255)
    else:
        rgb = (128, 128, 128)  # cinza
    
    # Criar imagem
    img = PILImage.new('RGB', (width, height), rgb)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    img_byte_arr.seek(0)
    return img_byte_arr

def upload_image_with_mock_user(color="red", description="Test image", tags="test,mock,user"):
    """Faz upload de uma imagem usando o mock user configurado"""
    # Criar imagem de teste
    img_bytes = create_test_image(color)
    
    # Preparar arquivo para upload
    files = {
        'file': (f"{color}_test.jpg", img_bytes.getvalue(), 'image/jpeg')
    }
    
    # Dados de formulário
    data = {
        'description': description,
        'tags': tags,
        'alt_text': f"Uma imagem de teste {color}",
        'team_id': MOCK_CONFIG['team_id']  # Importante: incluir team_id explicitamente
    }
    
    # Fazer upload
    result = make_api_request("POST", "/api/v1/images", files=files, data=data)
    
    # Exibir a imagem se houver URL de acesso
    if isinstance(result, dict) and "access_url" in result:
        print("\nImagem:")
        display(HTML(f'<img src="{result["access_url"]}" width="300" />'))
    
    return result

## 6. Atualizar Configuração do Mock User com Team ID Válido

Use esta célula para atualizar o `team_id` do mock user para um valor válido do banco de dados.

In [16]:
# Atualize o MOCK_CONFIG com um team_id válido
# Você pode selecionar um dos IDs listados acima

# Por exemplo, se o primeiro time da lista for válido:
if valid_teams:
    MOCK_CONFIG['team_id'] = valid_teams[0]['id']
    print(f"MOCK_CONFIG atualizado com team_id: {MOCK_CONFIG['team_id']}")
else:
    print("AVISO: Não foram encontradas equipes válidas. Configure o team_id manualmente.")
    # Você pode definir manualmente um team_id conhecido:
    # MOCK_CONFIG['team_id'] = "11111111-1111-1111-1111-111111111111"

# Exibir configuração atual
print("\nConfiguração atual do mock user:")
print(json.dumps(MOCK_CONFIG, indent=2))

NameError: name 'valid_teams' is not defined

## 7. Testar Upload de Imagem com Mock User

In [18]:
# Fazer upload de uma imagem vermelha
print("Fazendo upload de imagem vermelha...")
red_result = upload_image_with_mock_user("red", "Imagem vermelha de teste", "vermelho,teste,mock")

Fazendo upload de imagem vermelha...
Requisição: POST http://localhost:8080/api/v1/images
Usando mock user: daniel.schulz_computer_vision_team (Team ID: 2e07cc40-68e2-40cc-b686-cb05dce6c408)
Status: 500
Resposta:
{
  "detail": "Internal server error"
}


In [ ]:
# Fazer upload de uma imagem verde
print("Fazendo upload de imagem verde...")
green_result = upload_image_with_mock_user("green", "Imagem verde de teste", "verde,teste,mock")

In [ ]:
# Fazer upload de uma imagem azul
print("Fazendo upload de imagem azul...")
blue_result = upload_image_with_mock_user("blue", "Imagem azul de teste", "azul,teste,mock")

## 8. Listar Imagens Carregadas

In [ ]:
# Listar todas as imagens
print("Listando todas as imagens...")
all_images = make_api_request("GET", "/api/v1/images")

## 9. Resumo e Próximos Passos

Se a solução acima não funcionar, pode ser necessário modificar o arquivo `auth.py` no servidor para aceitar o cabeçalho `X-Mock-User`. Consulte o código de exemplo na seção 4 deste notebook.

Outra alternativa é criar um endpoint personalizado no servidor que aceita explicitamente um `user_id` e `team_id` para upload.